In [1]:
# import modules
from datetime import datetime, tzinfo, timedelta
import hdmf._version
from dateutil.tz import tzlocal
import platform
import math
import numpy as np
import uuid
import os
from os import environ
import scipy.io as sio
from hdmf.backends.hdf5.h5_utils import H5DataIO
import pandas as pd

#https://pypi.org/project/pyabf/
import pyabf

import pynwb
print('Using pynwb v%s'%pynwb.__version__)



Using pynwb v2.3.1


## Overview of all data sets

In [2]:
os. getcwd()

'C:\\Users\\Daksh Rathore\\INCF_Project'

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline 

# Studying the variables 

In [4]:
rn1_mat = sio.loadmat('Stimulus/rn1-matrix.mat')
rn1_par = sio.loadmat('Stimulus/rn1-param.mat')

In [5]:
site1 = sio.loadmat('Data/Site1-rn1.mat')

In [6]:
key_set = sorted(rn1_mat.keys())
key_set

['__globals__', '__header__', '__version__', 'stim_mat']

In [7]:
print('Contents of MAT file')
key_set = sorted(site1.keys())
key_set

Contents of MAT file


['__globals__', '__header__', '__version__', 'spk', 'trigger']

In [8]:
s1_spk = site1['spk']

In [9]:
#dtype=[('chan', '0'), ('model'), ('depth', '2'), ('position', '3'), ('stim', '4'), 
#('header'), ('spiketimes', '6'), ('outliers'), ('fs', '8'), ('probechan', '9'), 
#('meanwaveform', '10')])

## Creating a file

In [10]:
# random UUID for globally unique tag
unique_identifier = str(uuid.uuid4())

In [11]:
start_time = datetime.now(tz=tzlocal())
create_date = datetime.now(tz=tzlocal())

In [12]:
description = "High-density extracellular recordings from the primary auditory cortex in anesthetized rats listening to dynamic broadband stimuli"
notes = "High-density extracellular recordings from the primary auditory cortex in anesthetized rats listening to dynamic broadband stimuli "

nwbfile = pynwb.NWBFile(
            session_description='Example with various datatypes',
            identifier=unique_identifier,
            session_start_time=start_time,
            file_create_date=create_date,
            notes=notes,
            experimenter='Jermyn Z. See, Craig A. Atencio, Christoph E. Schreiner ', 
            experiment_description=description,   
)

## Creating A Device

In [13]:
device = pynwb.file.Device('Electrode')
nwbfile.add_device(device)

Electrode pynwb.device.Device at 0x2492920874512

## Electrode Group

In [14]:
nwbfile.add_electrode_column(name='probechan',description='Probe Channel')

In [15]:
spike_times = []
electrode_positions = []
electrode_depth = []
probe_channels = []
for i in range(s1_spk.shape[1]):
    unit_spk_times = s1_spk[0,i]['spiketimes'] / 1000.0  # convert to seconds
    spike_times.append(unit_spk_times)
    electrode_positions.append(s1_spk[0,i]['position'].tolist()[0])
    probe_channels.append(s1_spk[0,i]['probechan'].tolist()[0])
    electrode_depth.append(s1_spk[0,i]['depth'].tolist()[0])

In [16]:
elecs = np.concatenate((probe_channels,electrode_positions),axis=1)

In [17]:
un_elecs = np.unique(np.concatenate((probe_channels,electrode_positions,electrode_depth),axis=1), axis=0)

In [18]:
for i in range(len(un_elecs)):
    eg = nwbfile.create_electrode_group(name='rn1_Channel%s'%un_elecs[i,0],description="_",location="_",device=device) # maybe add position
    nwbfile.add_electrode(probechan=un_elecs[i,0],x=float(un_elecs[i,1]),y=float(un_elecs[i,2]),z=float(un_elecs[i,3]),group=eg,location="_")

## Spike Event Series

In [19]:
site1['spk'].shape

(1, 31)

In [20]:
spk = site1['spk']

In [21]:
import numpy as np
from pynwb import NWBFile, NWBHDF5IO
from pynwb.ecephys import SpikeEventSeries
from pynwb.file import DynamicTableRegion

for i in range(2):
    

    row_index = np.where((un_elecs == ((np.concatenate((spk[0,i]['probechan'],spk[0,i]['position'],spk[0,i]['depth']),axis=1)).flatten())).all(axis=1))[0]
    
    name = 'SingleUnit%s'%i

    conversion = 0.000001

    
    spikedata = np.isin(np.array(np.arange(1,1197265)),spk[0,i]['spiketimes']).astype(dtype='float64')

    electrodes = nwbfile.create_electrode_table_region(region=row_index.tolist(),description="LMAO",name='electrodes')

    spike_event_series = SpikeEventSeries(name=name+'_SpikeEvents',
                                      data=spikedata,
                                      timestamps=np.array(np.arange(1,1197265))/(spk[0,i]['fs']).tolist()[0],
                                      electrodes=electrodes,
                                      conversion=conversion,
                                      description='Snapshots of spike events for a single unit')
    spikewave = pynwb.ecephys.EventWaveform(spike_event_series, name=name+'_SpikeEventsInt')
    
    
    
    spike_event_waveform1 = SpikeEventSeries(name=name+'_SpikeWaveformMean',
                                      data=s1_spk[0,i]['meanwaveform'][:,1],
                                      timestamps=np.array(np.arange(1,len(spk[0,i]['meanwaveform'])+1))/(spk[0,i]['fs']).tolist()[0],
                                      electrodes=electrodes,
                                      conversion=conversion,
                                      description='Spike mean waveform of the unit')
    spikewaveform = pynwb.ecephys.EventWaveform(spike_event_waveform1, name=name+'_SpikeWaveformMean')
    
    nwbfile.add_acquisition(spikewave)
    
    nwbfile.add_acquisition(spikewaveform)
    
    

In [22]:
trig_data =  np.isin(np.array(np.arange(1,1197265)),np.array(site1['trigger'].tolist()[0])).astype(dtype='float64')
ts1 = pynwb.base.TimeSeries(name='StimTrig',data=site1['trigger'][0,:],starting_time=0.,rate=20000.,unit='Trigger')
nwbfile.add_stimulus(ts1)

for i in range(2):    

    chann = spk[0,i]['chan'].tolist()[0]
    
    name = 'SingleUnit%s'%i

    arr_rounded = np.round((rn1_mat['stim_mat'][chann]).astype(dtype='float64'), decimals=3)
    timeseries = pynwb.base.TimeSeries(name=name+'_AcquisitonChannel%s'%chann,data=arr_rounded,starting_time=0.,rate=20000.,unit='volt')
     
    nwbfile.add_stimulus(timeseries)   

## Writing to file

In [23]:
nwb_file_name = 'MyData.nwb'


io = pynwb.NWBHDF5IO(nwb_file_name, mode='w')
io.write(nwbfile)
io.close()
print("Created: %s"%nwb_file_name)

Created: MyData.nwb


In [24]:
import sys
sys.path.append("../NWB")

import nwb_info

nwb_file_name = 'MyData.nwb'
nwb_info.print_info(nwb_file_name, verbose=False)

NWB info

Info on MyData.nwb (57736620 bytes; modified: Wed Mar 29 01:53:28 2023)
    Attr   .specloc =	 <HDF5 object reference>
    Attr   namespace =	 core
    Attr   neurodata_type =	 NWBFile
    Attr   nwb_version =	 2.6.0
    Attr   object_id =	 9f76d918-3685-4634-8e78-e1949baa8bfa
    Field  lab =	 ---
    Field  experimenter =	 [b'Jermyn Z. See, Craig A. Atencio, Christoph E. Schreiner ']
    Field  institution =	 ---
    Field  lab =	 ---
    Field  notes =	 b'High-density extracellular recordings from the primary auditory cortex in anesthetized rats listening to dynamic broadband stimuli '
Successfully read file with h5py v3.6.0

Loaded without explicitly referring to Silverlab extensions
Successfully opened file with pynwb v2.3.1
    name = root
    notes = High-density extracellular recordings from the primary auditory cortex in anesthetized rats listening to dynamic broadband stimuli 
    subject = None
    experimenter = ('Jermyn Z. See, Craig A. Atencio, Christoph E. Schr

root pynwb.file.NWBFile at 0x2492985483376
Fields:
  acquisition: {
    SingleUnit0_SpikeEventsInt <class 'pynwb.ecephys.EventWaveform'>,
    SingleUnit0_SpikeWaveformMean <class 'pynwb.ecephys.EventWaveform'>,
    SingleUnit1_SpikeEventsInt <class 'pynwb.ecephys.EventWaveform'>,
    SingleUnit1_SpikeWaveformMean <class 'pynwb.ecephys.EventWaveform'>
  }
  devices: {
    Electrode <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    rn1_Channel10 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel12 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel13 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel14 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel21 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel22 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel23 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel24 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel25 <class 'pynwb.ecephys.ElectrodeGroup'>,
    rn1_Channel26 <class 'pynwb.ecephy